# Configuration

In [3]:
!pip install pydicom
!pip install optuna
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00


In [4]:
import pydicom
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import optuna
from scipy.ndimage import zoom
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt


from google.colab import drive

# Monter Google Drive avec force remount
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


# Chargement du dataset

In [5]:
import os
import torch
import pydicom
import numpy as np
from torch.utils.data import Dataset

def load_dicom_image(dicom_path):
    dicom_data = pydicom.dcmread(dicom_path)
    image = dicom_data.pixel_array.astype(np.float32)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))  # Normalisation
    return torch.tensor(image).unsqueeze(0)  # Ajouter une dimension channel (1, H, W)

class DICOMDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Dataset PyTorch qui charge des paires (image, masque).
        On suppose que les masques sont nommés "xxx_mask.dcm".
        """
        self.data_dir = data_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(data_dir) if f.endswith(".dcm") and "_mask" not in f])

        if len(self.image_files) == 0:
            raise FileNotFoundError(f"Aucun fichier DICOM trouvé dans {data_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.data_dir, self.image_files[idx])
        mask_path = image_path.replace(".dcm", "_mask.dcm")  # Assumer un fichier masque correspondant

        image = load_dicom_image(image_path)
        mask = load_dicom_image(mask_path) if os.path.exists(mask_path) else torch.zeros_like(image)  # Si pas de masque, mettre un 0

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return {"image": image, "mask": mask}



# Initialisation du dataset

dataset = DICOMDataset("/content/drive/MyDrive/Colab/ganglions_detection/dataset")
sample = dataset[0]
print(f"Image shape: {sample['image'].shape}, Mask shape: {sample['mask'].shape}")



✅ Séries DICOM chargée avec forme : torch.Size([706, 1, 512, 512])


# Prétraitement

# Modèle

## Création du modèle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [8]:
from monai.networks.nets import UNet
import torch

# Initialisation correcte de U-Net 3D
model = UNet(
    spatial_dims=3,   # Utilisation correcte pour un modèle 3D
    in_channels=1,    # Images en niveaux de gris
    out_channels=1,   # Segmentation binaire (1 canal de sortie)
    channels=(16, 32, 64, 128, 256),  # Nombre de filtres à chaque niveau
    strides=(2, 2, 2, 2),  # Strides pour le downsampling
    num_res_units=2,  # Nombre de blocs résiduels
).to(device)

print(model)


UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (adn): ADN(
            (N): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Se

In [10]:
from tqdm import tqdm

import torch.optim as optim
import monai.losses as monai_losses


# Paramètres d'entraînement
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Définition de la fonction de perte (Dice + BCE)
dice_loss = monai_losses.DiceLoss(sigmoid=True)
bce_loss = nn.BCEWithLogitsLoss()

def loss_function(y_pred, y_true):
    return dice_loss(y_pred, y_true) + bce_loss(y_pred, y_true)

# Optimiseur AdamW
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)


# Entraînement du modèle
def train_model(model, train_loader, val_loader, num_epochs=20):
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for batch in progress_bar:
            images, masks = batch['image'].to(device), batch['mask'].to(device)

            optimizer.zero_grad()
            outputs = model(images)

            loss = loss_function(outputs, masks)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}")

        # Validation
        validate_model(model, val_loader)

    print("Entraînement terminé ! 🚀")

# Fonction de validation
def validate_model(model, val_loader):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            images, masks = batch['image'].to(device), batch['mask'].to(device)
            outputs = model(images)
            loss = loss_function(outputs, masks)
            val_loss += loss.item()
    print(f"Validation Loss: {val_loss / len(val_loader)}")


In [27]:
batch_size = 4

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print(f"✅ DataLoaders créés : {len(train_loader)} batches d'entraînement, {len(val_loader)} de validation")


✅ DataLoaders créés : 177 batches d'entraînement, 177 de validation


## Entrainement du modèle

In [28]:
train_model(model, train_loader, val_loader, num_epochs=20)


RuntimeError: Given groups=1, weight of size [16, 1, 3, 3, 3], expected input[1, 4, 1, 512, 512] to have 1 channels, but got 4 channels instead

# Evaluation du modèle

In [ ]:
def evaluate_model_on_dicom(model, dicom_images, threshold=0.5):
    model.eval()
    dice_scores, iou_scores = [], []

    with torch.no_grad():
        outputs = model(dicom_images.to(device))
        outputs = torch.sigmoid(outputs)  # Appliquer la sigmoid pour obtenir des probas

        for i in range(len(dicom_images)):
            dice_scores.append(dice_score(outputs[i], dicom_images[i], threshold))
            iou_scores.append(iou_score(outputs[i], dicom_images[i], threshold))

    print(f"⚡ Dice Score moyen sur DICOM : {np.mean(dice_scores):.4f}")
    print(f"⚡ IoU moyen sur DICOM : {np.mean(iou_scores):.4f}")

# Lancer l'évaluation sur les images DICOM
evaluate_model_on_dicom(model, dicom_images)


#Optimisation du modèle

In [ ]:


def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-3)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Entraînement et évaluation
    return validation_loss  # Remplacez par votre métrique de validation

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)
print("Meilleurs hyperparamètres:", study.best_params)

# Prédiction du modèle

In [ ]:
def predict(model, input_volume):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(input_volume).unsqueeze(0).unsqueeze(0).float()
        segmentation, classification = model(input_tensor)
        return segmentation.squeeze().numpy(), classification.item()

# Exemple de prédiction
segmentation_map, classification = predict(model, volume_resized[0])
print("Classification:", "Pathologique" if classification > 0.5 else "Normal")
plt.imshow(segmentation_map, cmap='gray')
plt.show()